<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span><ul class="toc-item"><li><span><a href="#Таблица-contract" data-toc-modified-id="Таблица-contract-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Таблица contract</a></span></li><li><span><a href="#Таблица-personal" data-toc-modified-id="Таблица-personal-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Таблица personal</a></span></li><li><span><a href="#Таблица-internet" data-toc-modified-id="Таблица-internet-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Таблица internet</a></span></li><li><span><a href="#Таблица-phone" data-toc-modified-id="Таблица-phone-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Таблица phone</a></span></li></ul></li><li><span><a href="#Подключение-библиотек" data-toc-modified-id="Подключение-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подключение библиотек</a></span></li><li><span><a href="#Таблица-contract" data-toc-modified-id="Таблица-contract-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Таблица contract</a></span><ul class="toc-item"><li><span><a href="#Добавим-столбец-target" data-toc-modified-id="Добавим-столбец-target-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Добавим столбец target</a></span></li><li><span><a href="#Первичный-осмотр" data-toc-modified-id="Первичный-осмотр-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Первичный осмотр</a></span></li></ul></li><li><span><a href="#Таблица-personal" data-toc-modified-id="Таблица-personal-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Таблица personal</a></span></li><li><span><a href="#Таблица-internet" data-toc-modified-id="Таблица-internet-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Таблица internet</a></span></li><li><span><a href="#Таблица-phone" data-toc-modified-id="Таблица-phone-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Таблица phone</a></span></li><li><span><a href="#Сравнение-распределения-величин-ежемесячных-платежей-(MonthlyCharges)-всех-клиентов-и-тех,-кто-ушёл" data-toc-modified-id="Сравнение-распределения-величин-ежемесячных-платежей-(MonthlyCharges)-всех-клиентов-и-тех,-кто-ушёл-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Сравнение распределения величин ежемесячных платежей (MonthlyCharges) всех клиентов и тех, кто ушёл</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Сравнение-основных-статистик-ежемесячных-платежей-(MonthlyCharges)-всех-клиентов-и-тех,-кто-ушёл" data-toc-modified-id="Сравнение-основных-статистик-ежемесячных-платежей-(MonthlyCharges)-всех-клиентов-и-тех,-кто-ушёл-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Сравнение основных статистик ежемесячных платежей (MonthlyCharges) всех клиентов и тех, кто ушёл</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Сравнения-средних-обоих-групп" data-toc-modified-id="Сравнения-средних-обоих-групп-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Сравнения средних обоих групп</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Сравнения-гистограмм" data-toc-modified-id="Сравнения-гистограмм-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Сравнения гистограмм</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Доля-пользователей-телефонной-связи" data-toc-modified-id="Доля-пользователей-телефонной-связи-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Доля пользователей телефонной связи</a></span><ul class="toc-item"><li><span><a href="#Ушедшие" data-toc-modified-id="Ушедшие-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Ушедшие</a></span></li><li><span><a href="#Все" data-toc-modified-id="Все-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Все</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Доля--интернет-пользователей" data-toc-modified-id="Доля--интернет-пользователей-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Доля  интернет-пользователей</a></span><ul class="toc-item"><li><span><a href="#Ушедшие" data-toc-modified-id="Ушедшие-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Ушедшие</a></span></li><li><span><a href="#Все" data-toc-modified-id="Все-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>Все</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-12.3"><span class="toc-item-num">12.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Заключение" data-toc-modified-id="Заключение-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Заключение</a></span></li></ul></div>

## Data class